<img src='https://hammondm.github.io/hltlogo1.png' style="float:right">
Linguistics 531<br>
Fall 2025<br>
Jackson

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted, unless you have one of the excusable circumstances outlined in the syllabus (you or someone you need to care for are ill, you have a personal or family emergency, etc).
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class. (For grading, it needs to be able to run on my machine, and the way to do that is to limit yourself to the modules we've discussed and that are loaded into the Notebook.)
1. **Don't use editors *other* than Jupyter Notebook** to work on and submit your assignment, since they will mangle the autograding features: Google Colab, or even just editing the `.ipynb` file as a plain text file. Diagnosing and fixing that kind of problem takes a lot of my time, and that means less of my time to offer constructive feedback to you and to other students.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates, or the work of someone on Stack Overflow, or the work of a generative AI model. Using someone else's code and simply changing variable or object names is *not* doing your own work.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error. Code that runs with errors will get no credit from the autograder.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

my name: *\<Riley Thompson\>*

people I talked to about the assignment: *\<n/a\>*

# Homework #5

**This is due Tuesday, November 18, 2025 at noon (Arizona time).**

This assignment continues with the `NewB` corpus (downloadable [here](https://github.com/JerryWei03/NewB)).

imports:

In [1]:
import re
from math import isclose

# Used in the cosimfreq() implementation from class:
import numpy as np
#  You're free to use it here in your functions, but it's not a requirement

# As with last week, this might make your life much easier
from collections import Counter

In [2]:
!python --version

Python 3.11.2


**As before, this section is for autograding:**

What I need on my machine to properly grade this:

In [3]:
# Path on my own machine, needed for GRADING
newbfile = '/home/ejackson1/Downloads/linguistics/NewB/train_orig.txt'

# ie, DON'T CHANGE THIS CELL, CHANGE THE ONE BELOW!
#  If you change *this* cell, the autograding is likely to break.

*In the editable cell below, enter the path on your own machine,* then uncomment that line so the notebook works on your machine.

**BEFORE YOU SUBMIT to D2L, remember to comment out *your* path again.**

In [4]:
# YOUR path
#newbfile = '/home/rthompson/arizona/hlt531/NewB/train_orig.txt'

We're going to build a tf-idf document index from the first 50,000 lines of the `train_orig.txt` file. We will *not* stem or remove stop words.

**1.** The first step is to build a term-document index and a document-term index. (8 points)

Follow the guidelines in the docstring below and in the following assert statements. As with previous assignments, you'll create a text processing function as part of this, and when you read in the file, retain only upper and lower case ASCII letters, numbers, and the percent sign. Remember that the document ID is *not* the same as the publication source code.

You should be able to re-use your code from previous assignments here. Note that what's new this week is (1) we've added a limit to how many lines of the input file we use, and (2) we're creating both a term-document index and a document-term index.

In [5]:
def makeIndices(filepath,maxcount=50000):
    '''create document and term indices for the NewB corpus
    
    args:
        filepath (str): location of train_orig.txt
        maxcount (int): maximum number of lines
    
    returns:
        wordIndex (dict[str, List[ Tuple[int, int]]]): 
            a dictionary from terms (as strings) to
            a list of tuples of the form (doc-id, count)
        docIndex (List[Tuple[int, List[str], Set[Tuple[str, int]]]]): 
            a list of tuples of the form
               publication code (as an integer),
               tokenized text of document (as a list of strings),
               document vector (as a set of tuples: (word,count)
    '''
    # YOUR CODE HERE
    with open(filepath, "r") as file:
        lines = [line.rstrip("\n") for line in file]
    docID = 0
    docIndex = []
    wordIndex = {}
    if len(lines) > maxcount:
        lines = lines[:maxcount]
    for line in lines:
        id = int(line[0] if line[1] in " \t" else line[0:2])
        text = text_prep(line[2:])
        words = (Counter(text).items())
        
        for word in words:
            if word[0] in wordIndex:
                wordIndex[word[0]].append((docID,word[1]))
            else:
                wordIndex[word[0]] = [(docID,word[1])]
        docID += 1
        
        docIndex.append((id,text,set(words)))
        
        
    return wordIndex, docIndex
    
def text_prep(input):
    '''performs text normalization and tokenization on an input string
    
    Our process: anything that is not a letter (upper or lower case
        ASCII letters), digit (0-9), or the percent sign (%) is converted
        to space, and then terms are split on whitespace.
    
    args:
        input (str): a string of unprocessed text
    returns:
        output (List[str]): a list of strings of normalized tokens
    '''
    # YOUR CODE HERE
    return re.sub(r"[^a-zA-Z0-9%]"," ",input).split()



You've probably had enough testing of your `text_prep()` function that it should be working fine now. I won't include separate testing here, but bear in mind that if you are having trouble passing the tests below, it may be due to a problem in your `text_prep()` function. If so, the "practice" tests for that function from Homework 4 may be useful to you.

In [6]:
widx,didx = makeIndices(newbfile)

# test 1a, 1 pt
assert type(widx) == dict

In [7]:
# test 1b, 1 pt
# This one will be *very* sensitive to your text_prep() function
assert len(widx) == 32204

In [8]:
# test 1c, 1 pt
#  An indication of what your word index should be like
assert widx['apple'] == [(3984, 4),(10197, 1),(10604, 1),
 (27913, 1),(31235, 1),(33494, 1),(33511, 1),(46867, 1),
 (48326, 1),(48888, 1)]

In [9]:
# test 1d, 1 pt
assert type(didx) == list

In [10]:
# test 1e, 1 pt
assert len(didx) == 50000

In [11]:
# test 1f, 1 pt
#  These three tests are an indication of what your doc index should be like
assert didx[30861][0] == 1

In [12]:
# test 1g, 1 pt
assert didx[30861][1] == ['trump',  'holds',  'the',  'ground',  'lease',  'the',
                          'lease',  'for',  'the',  'land',  'on',  'which',  'the',
                          'building',  'stands']

In [13]:
# test 1h, 1 pt
assert didx[30861][2] == {('building', 1),  ('for', 1),  ('ground', 1),  ('holds', 1),
  ('land', 1),  ('lease', 2),  ('on', 1),  ('stands', 1),  ('the', 4),  ('trump', 1),
  ('which', 1)}

**2.** Use the term-document index to calculate idf values (an intermediate step on the way to ***tf*-idf** values) and return a dictionary from terms to idf values. (4 points)

Again, follow the guidance of the docstring and the following assert statements.

In [14]:
from math import log10
def makeIdfs(wordindex,maxcount=50000):
    '''calculate the idf value for each term in a word index
    
    args:
        wordindex (dict[str, List[ Tuple[int, int]]]): 
            a term-document index as created by makeIndices()
        maxcount (int): total number of documents to process;
            (by default, as set here, this is 50000; you don't
            need to re-check or re-assign this in your code.
            See https://realpython.com/python-optional-arguments/#using-python-optional-arguments-with-default-values )
        
    returns:
        (dict[str, float])
        a dictionary of each word in the collection
            and its idf value
    '''
    # YOUR CODE HERE
    N = maxcount  # total number of documents
    
    idfs = {}
    
    for term, postings in wordindex.items():
        dft = len(postings)  # number of documents containing the term
        
        if dft == 0:
            idf = 0
        else:
            idf = log10(N / dft)
        
        idfs[term] = idf
    
    return idfs
        
        

In [15]:
idfx = makeIdfs(widx)

# test 2a, 1 pt
assert type(idfx) == dict

In [16]:
# test 2b, 1 pt
# There should be an entry here for every term in your term-document index
assert len(idfx) == 32204

In [17]:
# test 2c, 1 pt
# I get 4.698970004336019
assert isclose(idfx['canon'],4.69897,abs_tol=0.0001)

In [18]:
# test 2d, 1 pt
#I get 2.11690664142431
assert isclose(idfx['help'],2.11690,abs_tol=0.0001)

As we would expect, frequent terms (like function words) have very low idf values.

Because of the nature of this collection, certain other terms also have very low idf scores, even though they're not function words or common stop words, and would in other collections likely have ***higher*** idf scores. (So, note that idf scores will *really* depend on your data set!)

Here are the 20 terms with lowest idf scores; note that "donald" and "trump" are among them. *Before moving on, make sure you understand why that is!*

In [19]:
sorted([(idfx[key], key) for key in idfx])[:20]

[(0.0, 'trump'),
 (0.22883893956150084, 'the'),
 (0.3805727251185078, 'to'),
 (0.4324090650491771, 'a'),
 (0.450457272115119, 'and'),
 (0.45859550737778554, 'of'),
 (0.5042059802251323, 'in'),
 (0.6440124083236123, 'that'),
 (0.6946916321358564, 'his'),
 (0.7188302265902644, 'on'),
 (0.7319839792180809, 'for'),
 (0.770471736212328, 'donald'),
 (0.7723189272471299, 'he'),
 (0.8082137524178006, 'said'),
 (0.8260565627156234, 'is'),
 (0.835290336460121, 'with'),
 (0.9376436819145622, 'has'),
 (0.9437051129237721, 'was'),
 (0.9510146974292889, 'as'),
 (1.0061230850587888, 'at')]

**3.** We now build a tf-idf index from the document-term index and the idf values. (6 points)

*HINT: This is really an index with precisely the same structure as our document-term index, but simply with different numerical values--floats, not just integer counts--in each document vector.*

In [20]:
def makeTfidf(idfs,docindex):
    '''creates a tfidf document-term index from idf scores
    and a document index
    
    args:
        idfs (dict[str, float]):
            idf scores (as a dictionary) as created by makeIdfs()
        docindex (List[Tuple[int, List[str], Set[Tuple[str, int]]]]): 
            a document index as created by makeIndices()
    returns:
        (List[Tuple[int, List[str], Set[Tuple[str, float]]]])
        tf-idf index as a list where the list
            index corresponds to the docID
            and each entry is a triple:
                publication code
                document text
                set of pairs:
                    word
                    tfidf score
    '''
    # YOUR CODE HERE
    tfidf_index = []

    for id, text, terms in docindex:

        tfidf_set = set()

        for term, tf in terms:
            idf = idfs.get(term, 0)
            tfidf = tf * idf
            tfidf_set.add((term, tfidf))

        tfidf_index.append((id, text, tfidf_set))

    return tfidf_index
            
    

In [21]:
tfidfidx = makeTfidf(idfx,didx)

# test 3a, 1 pt
assert type(tfidfidx) == list

In [22]:
# test 3b, 1 pt
assert len(tfidfidx) == 50000 and tfidfidx[49000][0] == 2

In [23]:
# test 3c, 1 pt
assert tfidfidx[49000][1] == ['donald', 'trump', 'didnt', 'tell', 'the', 'truth', '83', 'times', 'in', '1', 'day']

In [24]:
# Here's a document we'll look at
print("Sentence:\n",tfidfidx[41584][1],'\n')
print("Raw counts:\n",sorted(didx[41584][2]),'\n')
print("tfidf:\n",sorted(tfidfidx[41584][2]))

Sentence:
 ['the', 'copying', 'of', 'the', 'letter', 'to', 'the', 'justice', 'department', 'attracted', 'wide', 'notice', 'in', 'washington', 's', 'close', 'knit', 'election', 'law', 'bar', 'as', 'did', 'the', 'claim', 'in', 'the', 'lawsuit', 'that', 'the', 'use', 'of', 'the', 'trump', 'foundation', 'to', 'benefit', 'the', 'trump', 'campaign', 'was', 'willful', 'and', 'knowing'] 

Raw counts:
 [('and', 1), ('as', 1), ('attracted', 1), ('bar', 1), ('benefit', 1), ('campaign', 1), ('claim', 1), ('close', 1), ('copying', 1), ('department', 1), ('did', 1), ('election', 1), ('foundation', 1), ('in', 2), ('justice', 1), ('knit', 1), ('knowing', 1), ('law', 1), ('lawsuit', 1), ('letter', 1), ('notice', 1), ('of', 2), ('s', 1), ('that', 1), ('the', 8), ('to', 2), ('trump', 2), ('use', 1), ('was', 1), ('washington', 1), ('wide', 1), ('willful', 1)] 

tfidf:
 [('and', 0.450457272115119), ('as', 0.9510146974292889), ('attracted', 3.3767507096020997), ('bar', 3.0087739243075053), ('benefit', 2.703

In [25]:
# I'm going to put the tf-idf representation of the document into a Counter so it's
#  easier to pull values out for a specific term (using the Counter like a dict)
res = Counter(dict(tfidfidx[41584][2]))

# test 3d, 1 pt
# I get 0 and 4.221848749616356
assert isclose(res['trump'],0.,abs_tol=0.0001) and isclose(res['willful'],4.2218,abs_tol=0.0001)
# I get 0.9171910147555711 and 1.8307115164920067
assert isclose(res['of'],0.91719,abs_tol=0.0001) and isclose(res['the'],1.83071,abs_tol=0.0001)

In [26]:
# Here's another document that we'll look at
print(didx[43272][1])

['journalism', 'somehow', 'managed', 'to', 'get', 'the', 'trump', 'phenomenon', 'wrong', 'wrong', 'wrong', 'while', 'writing', 'more', 'and', 'more', 'about', 'trump', 'trump', 'trump']


In [27]:
res = Counter(dict(tfidfidx[43272][2]))

# test 3e, 1 pt
# I get 2.8459360287479374 and 7.517536217944672
assert isclose(res['more'],2.8459,abs_tol=0.0001) and isclose(res['wrong'],7.5175,abs_tol=0.0001)

In [28]:
# Here's another document that we'll look at
print(didx[7328][1])

['but', 'when', 'you', 'have', 'a', 'fight', 'you', 'get', 'sinatra', 'you', 'get', 'donald', 'trump', 'you', 'get', 'lee', 'iacocca', 'and', 'you', 'get', 'dr']


In [29]:
res = Counter(dict(tfidfidx[7328][2]))

# test 3f, 1 pt
# I get 8.040034638701437 and 7.415487857287048
assert isclose(res['you'],8.0400,abs_tol=0.0001) and isclose(res['get'],7.4155,abs_tol=0.0001)

We'll need the `cosimfreq()` function again:

In [30]:
#cosine similarity wrt/frequencies or tfidfs
def cdot(count1, count2):
    """dot product for two vectors as Counters"""
    return sum(count1[word]*count2[word] for word in count1)

def clength(count1):
    """length of a vector as Counter"""
    return np.sqrt(sum(count1[word]**2 for word in count1))

def cosimfreq(d1,d2):
    # gotta get 'em from lists of tuples into Counters
    d1, d2 = Counter(dict(d1)), Counter(dict(d2))
    d1len, d2len = clength(d1), clength(d2)
    denom = d1len * d2len
    if denom == 0: return 0
    return float(cdot(d1, d2))/float(denom)

**4.** Write a search function for the tf-idf index from question 3 that uses only cosine similarity (as given just above). (5 points)

The function should have this argument structure:

```python
search(query, index, idfs)
```

The function should return a rank-ordered list of the 10 **best** matches.

*Hint: Note that the idf scores, as well as the tf-idf index and query, are being passed to this search function. Consider the last several homework assignments: often you made a change to the structure of an index, which meant that you needed to make the same change to the distance functions and to the search function. What did you have to update in the search functions before?*

In [31]:
def search(query,index,idfs):
    '''search a tf-idf index for documents that are most similar
       to a query
       (This time we won't offer a choice of distance function,
       but will just use cosine similarity.)
    
    args:
        query (str): 
            search query (as a string)
        idx (List[Tuple[int, List[str], Set[Tuple[str, float]]]]):
            tf-idf index from makeTfidf()
        idfs (dict[str, float]): 
            idf scores from makeIdfs()
    returns:
        (List[Tuple[float, int]])
        10 best matches as a list of tuples:
            cosine similarity score
            document index
    '''
    
    qwords = text_prep(query)
    q_tf = Counter(qwords)

    q_tfidf = { term: tf * idfs.get(term, 0) for term, tf in q_tf.items() }

    q_vector = list(q_tfidf.items())

    results = []
    for docID, (pubcode, text, tfidf_set) in enumerate(index):

        d_vector = list(tfidf_set)

        sim = cosimfreq(q_vector, d_vector)

        results.append((sim, docID))

    results.sort(key=lambda x: x[0], reverse=True)

    return results[:10]

In [32]:
res1 = search("melania is trump's wife",tfidfidx,idfx)

# test 4a, 1 pt
# Make sure you're getting the structure right
assert isinstance(res1,list) and len(res1) == 10 \
    and all(isinstance(res1[i],tuple) for i in range(10)) \
    and all(isinstance(res1[i][0], float) for i in range(10)) \
    and all(isinstance(res1[i][1], int) for i in range(10)) \
    and all(res1[i][0] >= res1[i+1][0] for i in range(9))  # note >= since we might have ties

In [33]:
# test 4b, 1 pt
# Make sure you're getting the right results returned
assert res1[0][1] == 29285 and res1[1][1] == 41827

In [34]:
# As a sanity check, here are your results:
[(result,' '.join(didx[result[1]][1])) for result in res1]

[((0.849969028092293, 29285), 'trump and his wife melania'),
 ((0.6215127702787681, 41827), 'trump and his wife melania hosted mr'),
 ((0.518398074561728, 46573),
  'full text here is a look at the life of melania trump wife of president donald trump'),
 ((0.5083112011174595, 21175),
  'trump is 24 years older than his wife melania'),
 ((0.5043656908492963, 10060),
  'from left trump his son barron trump and wife melania trump'),
 ((0.5013248622362927, 32775),
  'trump and his wife melania who was pregnant at the time'),
 ((0.4673037889163044, 15962),
  'trump was accompanied by his wife melania who introduced him'),
 ((0.45156314335095366, 3499),
  'he then introduced his wife melania who said trump will work for all americans'),
 ((0.45156314335095366, 21510),
  'he then introduced his wife melania who said trump will work for all americans'),
 ((0.4364630422890691, 47305),
  'full text here is a look at the life of melania trump wife of 45th us president donald trump')]

In [35]:
# test 4c, 1 pt
# Make sure you're getting the right similarity values, too
# For the top two, I get 0.849969028092293 and 0.6215127702787681
assert isclose(res1[0][0],0.84997,abs_tol=0.0001) and isclose(res1[1][0],0.62151,abs_tol=0.0001)

In [36]:
res2 = search('trump lives in the white house',tfidfidx,idfx)

# test 4d, 1 pt
assert res2[0][1] == 22752 and res2[4][1] == 45145

In [37]:
# Sanity check
[(result,' '.join(didx[result[1]][1])) for result in res2]

[((0.6319877962507022, 22752), 'trump in the white house'),
 ((0.6158608687047019, 11090), 'the trump white house'),
 ((0.6067586288920347, 31782), 'trump to the white house'),
 ((0.5931508697856988, 37704), 'trump is in the white house'),
 ((0.49816997165817267, 45145), 'trump and the white house said mr'),
 ((0.4539447710242628, 13798), 'full text the trump white house'),
 ((0.42225789139265635, 15188), 'he lives on the phone walker said of trump'),
 ((0.40932614931370953, 2490), 'we can go on with our lives ivana trump said'),
 ((0.4027002229591978, 6749), 'one of the characters lives in trump towers'),
 ((0.39402213680464887, 26481), 'trump was in the white house for a reason')]

In [38]:
# test 4e, 1 pt
# I get 0.3940221368046489 for the tenth result
assert isclose(res2[9][0],0.3940, abs_tol=0.0001)

**5.** Calculate the MAP score for your search function based on `res1` and `res2` above *by hand* (that is, step by step, typed below) and *show your work*. (2 points)

Note that this requires that you look up the sentence for each returned result and judge whether it was relevant to the query. Your choice of relevant or not isn't graded, so don't invest too much thought in what *ought* to be relevant or not; choose a judgment for each returned result, and then you'll need to properly calculate the MAP score based on those choices.

First, here is some text formatting code in Markdown for you to use. Copy the source of this cell into the answer cell (since this cell will be read-only), then give your relevance scores in this table. Code each result as "relevant" (Y) or "not relevant" (N) for its query&mdash;meaning, is this result a good result given that particular query.

As an example of what it means for one of our results to be "relevant" for the queries here, consider our `res2` query, "trump lives in the white house". Many of the results involve the terms "trump" and "white house", and to me they do seem like reasonable results for that query, but note that result 15188 is "he lives on the phone walker said of trump". To me, that seems like **not** a relevant result for "trump lives in the white house". However, the final decision is up to you, and I'm the one who has to make sure that the math is correct given *your* relevancy results.

&nbsp; | `res1` | `res2`
-------|--------|-------
0
1
2
3
4
5
6
7
8
9

Based on your table, once you generate it in the cell below, please show the step-by-step calculation of the MAP score for this search system. You may find the following references helpful if you need to properly format any of your mathematical equations in LaTeX and Markdown:

https://jupyterbook.org/en/stable/content/math.html

https://ashki23.github.io/markdown-latex.html#latex

&nbsp; | `res1` | `res2`
-------|--------|-------
0|yes|yes
1|yes|yes
2|yes|yes
3|yes|yes
4|yes|no
5|yes|yes
6|yes|no
7|yes|no
8|yes|no
9|yes|yes


By my judgement all the documents returned in res1 are relevant, which means as a shortcut we know its average precision is 1.
The average precision for res2 will be more involved.

`rank` | `relevant?` |`relevant so far`| `precision`
-------|--------|-------|-------
0|yes|1|1/1
1|yes|2|2/2
2|yes|3|3/3
3|yes|4|4/4
4|no|4|
5|yes|5|5/6
6|no|5|
7|no|5|
8|no|5|
9|yes|6|6/10

so (1 + 1 + 1 + 1 + 5/6 + 6/10)/10 = .5333333

Our two precisions are 1 and 5.33333, therefore the mean average precision is

(1 + .533333)/2 = .7717

MAP = .7717

